In [3]:
from numpy import sin, cos, pi, array
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import matplotlib.animation as animation
import math


metadata = dict(title='Plane', artist='Pete Florence',
        comment='Fly time')
writer = animation.AVConvWriter(fps=20, metadata=metadata, bitrate=500)


# constant velocity
v = 1

def derivs(state, t):

    dqdt = np.zeros_like(state)
    
    dqdt[0] = -v * sin(state[2]) # Move origin, not plane (invert v to switch back)
    
    dqdt[1] = -v * cos(state[2]) # Move origin, not plane (invert v to switch back)

    dqdt[2] = .3 # constant for now, but later will be u

    return dqdt

# create a time array from 0...30 sampled at 0.05 second steps
dt = 0.05
t = np.arange(0.0, 5, dt)

# initial positions
# x and y are position of origin.  We move the origin, not the plane, so that the plane stays in the same spot in the plot.
# psi is yaw, and psidot is yawdot of plane
x = 0.0
y = 0.0
psi = -10.0

rad = pi/180.0

# initial state
state = np.array([x, y, psi*rad])

# integrate your ODE using scipy.integrate.
y = integrate.odeint(derivs, state, t)

xworld = y[:,0]
yworld = y[:,1]

x_obj1 = xworld + 5
y_obj1 = yworld + 3

x_obj2 = xworld - 4
y_obj2 = yworld + 7

In [ ]:
%matplotlib qt
plt.rcParams['figure.figsize'] = 10, 7
fig = plt.figure()
ax = fig.add_subplot(111, autoscale_on=False, xlim=(-15, 15), ylim=(-5, 20))
#ax.grid()

#FOV parameters
FOVangle = 130*math.pi/180
FOVlength = 10
theta_neg = -FOVangle/2
theta_pos = FOVangle/2
FOV_data = [FOVlength*math.cos(math.pi/2-theta_pos),FOVlength*math.sin(math.pi/2-theta_pos),FOVlength*math.cos(math.pi/2-theta_neg),FOVlength*math.sin(math.pi/2-theta_neg)]


im = plt.imread('deltawing.png')
newax = fig.add_axes([0.477, 0.15, 0.07, 0.1], anchor='SW')
newax.imshow(im)
newax.axis('off')
ax.set_aspect('equal', 'datalim')
        
FOV, = ax.plot([], [], '--' , lw=1, color='blue')
line, = ax.plot([], [], 'o', lw=2, color='red')
time_template = 'time = %.1fs'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)

def init():
    FOV.set_data([], [])
    line.set_data([], [])
    time_text.set_text('')
    return line, time_text

def animate(i):
    thisx = [xworld[i], x_obj1[i], x_obj2[i]]
    thisy = [yworld[i], y_obj1[i], y_obj2[i]]
    
    #thisx = [3, -4]
    #thisy = [4, 10]

    FOV.set_data([FOV_data[0],0,FOV_data[2]],[FOV_data[1],0,FOV_data[3]])
    line.set_data(thisx, thisy)
    time_text.set_text(time_template%(i*dt))
    return line, time_text

ani = animation.FuncAnimation(fig, animate, np.arange(1, len(y)),
    interval=20, blit=False, init_func=init)
ani.save('plane.mp4', fps=20, bitrate=5000, codec="h264", writer=writer)
plt.show()

IOError: [Errno 2] No such file or directory: 'deltawing.png'

In [5]:
import os
if os.path.exists(os.path.join(os.curdir, "plane_h264.mp4")):
    os.system("rm plane_h264.mp4")
os.system("avconv -i plane.mp4 -c:v h264 -c:a copy plane_h264.mp4")

0

In [6]:
import os
import io
import base64
from IPython.display import HTML

video = io.open('/Users/pflomacpro/GeometricPDEs/plane_h264.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))